In [1]:
# A simple notebook to quickly look at fields in a WACCM history file
# /glade/u/home/marsh/demo/quicklook.ipynb
# created: 18 Sep 2020 (drm)

# This notebook creates a widget that allows for quick inspection of the fields 
# in a CESM history file. It reads in the fields and presents them in a drop down menu.
# When a field is selected it creates a plot if the field is lat/lon or lat/lon/lev.  
# Options exist to plot a slice in latitude and longitude or latitude height at fixed 
# longitude or a zonal mean

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from ipywidgets import interactive

# open the WACCM netcdf file using xarray

data_dir = '/glade/u/home/marsh/demo/sample_data/'
waccm_file = 'b.e21.BWma1850.f19_g17.CMIP6-piControl-WACCM-MA-2deg.001.cam.h0.0300-12.nc'
ds = xr.open_dataset(data_dir+waccm_file)

# determine the size of the fields
ntime = ds['time'].size
nlon = ds['lon'].size
nlev = ds['lev'].size

In [2]:
# a function to create a contour plot up to a specified log pressure level

def plt_yz(a, logtop):
    a.where(ds.coords["lev"] > 10**logtop).plot.pcolormesh()
    ax = plt.gca()
    ax.set_ylim([1000,10**logtop])
    ax.set_yscale('log')
    ax.set_ylabel('Pressure (hPa)')
    
    return
    

In [3]:
# a function that responds to widget selections by changing the plot

def plt_field(field, time_index, level, lon_index, plot_type, logtop):
    plt.figure(figsize=(12,8))
    
    var = ds[field]
    
    if (var.dims == ('time','lev', 'lat', 'lon')):
        
        var1 = var.isel(time=time_index)
        
        if (plot_type == 'lon-lat'):
            var1[level,:,:].plot.imshow()
        elif (plot_type == 'zonal mean'):
            a = var1.mean(dim='lon')
            plt_yz(a, logtop)
        elif (plot_type == 'fixed lon'):
            a = var1[:,:,lon_index]
            plt_yz(a, logtop)
    elif (var.dims == ('time', 'lat', 'lon')):
        var[time_index,:,:].plot.imshow()
    else:
        plt.scatter((0,360),(-90,90))
        plt.text(180, 0, 'select a 2-D or 3-D field', horizontalalignment='center')
    
    plt.show()
    
    return

# create the widget options: field, time, level, longitude, plot type and extent

w = interactive(plt_field, field=ds.data_vars.keys(), 
          time_index = (0,ntime-1,1),
          level=(0,nlev-1,1),
          lon_index = (0,nlon-1,1),
          plot_type=['lon-lat', 'zonal mean', 'fixed lon'], 
          logtop=(-5,2,0.5))

# display the widget

output = w.children[-1]
output.layout.height = '500px'
display(w)

interactive(children=(Dropdown(description='field', options=('gw', 'zlon_bnds', 'hyam', 'hybm', 'P0', 'hyai', …